# Time-resolved scRNA-seq

## RNA velocity

Although the seminal RNA velocity work is exciting, it has the following limitations: 

<!-- 1. It can only predict short-term direction and magnitude of RNA dynamics.
2. It is mostly a descriptive instead of a predictive tool. 
3. It relies on the `mis-priming` of intron reads for current single-cell platforms and thus the intron measures are biased and inaccurate. 
4. RNA velocity was estimated as $U - \gamma / \beta S$ ($U$: unspliced RNA, $S$: spliced RNA, $\gamma$: degradation rate, $\beta$: splcing rate, $\gamma / \beta$ is the slope of the steady state cell fitting.), it is thus scaled by the splicing rate and lacks real physical meanings (i.e. molecules / hour).

We reason that metabolic labeling based method which measures both the historical or old, and the new and nascent RNA of cells in a controllable way will be better measurements for RNA velocity and transcriptomic dynamics. When extending metabolic labeling to single cell RNA-seq, labeling based scRNA-seq essentially measures two modalities or timepoints for the same cell. 
--> 

1. Because the intron reads are generated through mis-priming on polyA- or polyT- enriched intronic regions of nascent pre-RNA, it can be difficult to apply conventional RNA velocity to most transcription factors, which are typically expressed at low levels, and genes with no polyA/T-enriched intron regions;
2. Although many biological systems, including hematopoiesis, involve rapid but coordinated changes of RNA transcription rates for a considerable number of genes (Barile et al., 2021), existing estimation methods of splicing RNA velocity (Bergen et al., 2020; La Manno et al., 2018) all assume constant transcription rates ($\alpha_{on}$ for the induction phase, $\alpha_{off}$ for the repression phase) and often lead to nonsensical backward RNA velocity flow.
3. The linear regression methods used by conventional RNA velocity ignores the distribution of unspliced and spliced RNA, which can be used to improve the estimation of kinetic parameters;
4. For systems far away from the pseudo-steady state, using cells with extreme RNA expression levels for linear regression may lead to inaccurate velocity calculations for most cells;
5. The time scale for conventional RNA velocity ($v = u - \tilde{\gamma} s$) is scaled by $\beta$ (since $\tilde{\gamma}$ is the ratio of $\beta$  and $\gamma$), which makes the estimated velocity a relative quantity;
6. Conventional RNA velocity only estimates velocity for observed cells. Thus, it is a discrete, sparse, and local measure of cell dynamics and often merely used as a descriptive instead of a predictive tool. 


A great deal of effort has been devoted to improving conventional RNA velocity estimation (La Manno et al., 2018) in regard to challenges 3) and 4) and extend the concept to “protein velocity” (Gorin et al., 2020), but 1), 2), and 5) are fundamental limitations that cannot be resolved at the computational level without additional experimental information. In dynamo, we introduce our methods for analyzing conventional scRNA-seq data, addressing some of the issues with existing RNA velocity methods. More importantly, we also focus on computational methods for computing RNA velocity for metabolic labeling data, which reconciles the splicing- and labeling-based kinetics and overcomes other drawbacks of conventional RNA velocity methods. Finally, to address 6), we go beyond RNA velocity samples of single cells to map the continuous vector field functions in transcriptomic space and perform sophisticated differential geometry analyses to gain various functional vector field predictions and biological insights.



## How does metabolic labeling work

How can we quantify nascent RNA via metabolic labeling? Overall there are two different methods, the biotin purification or chemical conversion based approach. Both approaches are quiet similar in that we first need to applying different labeling strategies to label the cells. For biotin purification, we need to use thiol-specific biotinylation to tag labeled mRNA. Then the streptavidin beads can be used to pull down and separate the pre-exisiting RNA and newly transcribed RNA. Then we will follow by preparing two separate libraries, old and new RNAs, for sequencing. There are a few very well known issue regarding this method:

1. it often introduces 20-30% cross-contanimation between old and new RNAs,
2. it also leads to some normalization issues between different libraries. 


On the other hand, the chemical conversion based approaches avoid the labrious and error-prone procedure of separating old/old RNA and preparing two different libraries and emerged as the favored strategy recently. The key idea of chemical conversion based methods are that by some chemical reaction we can artificially introduce T to C mutation which can then be used to distinuigh labelled and thus new RNA from old RNA. There are about three different chemistry developed: IAA alkylation or hydrogen bond reconfiguration via TimeLapse-seq or TUC-seq chemistry. 

In fact, metabolic labeling has been widely adapted for the past a few decades. We can use various nucleotides to label RNA, for example, BrU, Eu and Biotin-NTP. For 4sU based labeling, there are about three different strategies, namely, SLAM-seq, TUC-seq, and Time-lapse-seq. 

<img src="https://user-images.githubusercontent.com/7456281/93838316-346af300-fc57-11ea-9cf9-79d37d8ff927.png">


## Metabolic labeling based scRNA-seq

Recently a few groups adapted the bulk method to either the plate-based scRNA-seq with SMART-seq2 method, for example, [scSLAM-seq](https://www.nature.com/articles/s41586-019-1369-y) or [NASC-seq](https://www.nature.com/articles/s41467-019-11028-9). [scEU-seq](https://science.sciencemag.org/content/367/6482/1151.full) is based on [CEL-Seq2](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-016-0938-8) and is also plate-based but uses UMI in contrast to scSLAM-seq or NASC-seq. The scEU-seq method is based on EU and required purification and it thus may involve cross-contanimation or normalization issues. 

[Cao, et al](https://www.nature.com/articles/s41587-020-0480-9#:~:text=Abstract,not%20directly%20capture%20transcriptional%20dynamics.&text=We%20used%20sci%2Dfate%20to,in%20%3E6%2C000%20single%20cultured%20cells) recently developed sci-fate which integrates 4sU labeling with combinatorial indexing based scRNA-seq, thus it can potentially enable measuring hundread thousands of single cells.  

For the first time, [Wu lab](https://www.wulabupenn.org/) from Upenn developed the RNA metabolic labeling argumented scRNA-seq based on drop-seq, scNT-seq. 
<img src="https://user-images.githubusercontent.com/7456281/93839221-4601ca00-fc5a-11ea-8e56-d39ec1725a6b.png">


## Comparison between different labeling based scRNA-seq methods


In [Qiu, Hu, et. al](https://www.nature.com/articles/s41592-020-0935-4), we performed a detailed comparison (Supplementary table 7) between scNT-seq with other available methods. Especially for the improved second-strand synthesis based strategy, we are able to obtain substantially high number of genes and UMIs per cell with relatively few number of reads. Thus scNT-seq is arguably one of the best metabolic labeling based scRNA-seq strategies. 

In our study, we show that dynamo can be used to leverage scNT-seq datasets for time-resolved RNA-velocity analysis. Those results demonstrate the power of dynamo and scNT-seq in revealing the fine-grained transcriptomic dynamics that cannot be captured by coventional methods based solely on splicing data. 
<img src="https://user-images.githubusercontent.com/7456281/93838287-1b624200-fc57-11ea-9674-76006ba07950.png">


## Labeling strategies 

We can be very creative and smart in designing the metabolic labeling experiments. We suggest three general labeling strategies, namely one-shot, kinetics/pulse, and degradation/ chase experiments, which cover most labeling methods in published tscRNA-seq experiments, aimed at estimating different RNA kinetic parameters (See figure below). It is possible to extend or combine these three general labeling strategies to more complicated labeling schemes, e.g., the fourth type in (See figure below), which consists of a time-series of multiple kinetics experiments, or a mixture experiment as in scEU-seq study (Battich et al., 2020), in which a variable initial kinetics experiment and later accompanying degradation experiment are conducted but kept in a fixed time period.

For example, you can design an experiment where you can take different days and perform a kinetics experiment at each day. This can help you obtain transcription rate, splicing and degradation rate over time. But this is often time-consuming, so we may just choose a typical day for a single kinetics experiment. In addition, we may also perform a degradation experiment where we label the cells with 4sU for an extended time period to saturate the 4sU labeling in cells. Then we can wash out the 4sU and replaced with excess U, followed by chasing at different time points. This can help us to estimate the splicing and degradation rates (and half life) of RNA. We can also just design a one-shot labeling experiment to label cells at different time points. Since splicing and degradation rate of mRNA can be reasonably assumed to be constants, when combining one-shot experiments with degradation experiments, we are able to get even more accurate estimates of the transcription rates at each time point. We also want to note that we can combine different labeling strategies, for exmple, combining pulse chase in a single experiment or integrating metabolic labeling with drug treatment or genetic perturbations. 

<img src="https://user-images.githubusercontent.com/7456281/93838322-392fa700-fc57-11ea-9019-e76358160f57.png">


## Dynamo's comprehensive model framework for analyzing lableing datasets


<!-- In order to fully take advantage of the scSLAM-seq data, we recently developed a sophisticated framework, dynamo that provides an inclusive model of expression dynamics with scSLAM-seq and multiomics, vector field reconstruction and potential landscape mapping. In dynamo, we abstract every steps from RNA transcription, splicing, metabolic labeling, translation and RNA or protein degradation. We can model the mean and variance of RNA species via a set of moment equations, we then transform them into a matrix format and solve them efficiently. In dynamo, we also implemented the traditional RNA velocity method based on the steady state assumptions to support analyzing regular 10 x data. Similarly, dynamo supports studying cite-seq data to estimate protein velocity. --> 

To develop a unified framework for extracting RNA dynamics information from cscRNA-seq and tscRNA-seq datasets, we constructed an inclusive model (Fig. 2A from [Qiu, et. al](https://www.biorxiv.org/content/10.1101/696724v2)) that considers on-off switching of promoter state, RNA metabolic labeling (when using tscRNA-seq data), RNA splicing and degradation, and even translation and protein degradation when transcriptomic– proteomic coassay data are available. To account for different data types and experiments, we further implement three reduced models: **Model 1** considers RNA transcription, splicing and degradation, but not RNA metabolic labeling, and is tailored for cscRNA-seq, whereas both **Models 2** and **3** are tailored for tscRNA-seq with metabolic labeling, with the difference that only **Model 3** considers RNA splicing (Fig. SI2A from [Qiu, et. al](https://www.biorxiv.org/content/10.1101/696724v2)).

When only cscRNA-seq data are available, or when one needs to use splicing data from tscRNA-seq experiments (which also capture intrinsic splicing kinetics), dynamo can be used to
estimate the relative degradation rate constant ($\hat{\gamma} = \gamma / \beta$) and relative spliced RNA velocity (Fig. 2B, top from [Qiu, et. al](https://www.biorxiv.org/content/10.1101/696724v2)). The estimation methods built upon **Model 1** from Fig. SI2A from [Qiu, et. al](https://www.biorxiv.org/content/10.1101/696724v2) include both the original method (La Manno et al., 2018) and the generalized method of moments (GMM). The GMM, in turn, consists of the stochastic splicing method, which relies on a master equation formulation of RNA kinetics (see MATERIALS AND METHODS) and is equivalent to the stochastic method developed recently (Bergen et al., 2020), and a new approach, the negative binomial (NB) method, which additionally models the gene expression at steady state as a negative binomial (NB) distribution, in the same vein as in previous studies (Grün et al., 2014).

By comparison, from a tscRNA-seq experiment, one can estimate the absolute kinetic parameters ($\alpha, \beta, \gamma$) and calculate absolute unspliced, spliced, new, or total RNA velocity, depending on the exact labeling strategy and underlying model used (Fig. 2B, bottom from [Qiu, et. al](https://www.biorxiv.org/content/10.1101/696724v2)). We suggest three general labeling strategies, namely **one-shot**, **kinetics/pulse**, and **degradation/chase** experiments, which cover most labeling methods in published tscRNA-seq experiments, aimed at estimating different RNA kinetic parameters (Fig. 2C from [Qiu, et. al](https://www.biorxiv.org/content/10.1101/696724v2)). It is possible to extend or combine these three general labeling strategies to more complicated labeling schemes, e.g., the fourth type in (Fig. 2C from [Qiu, et. al](https://www.biorxiv.org/content/10.1101/696724v2)), which consists of a time-series of multiple kinetics experiments, or a mixture experiment as in scEU-seq study (Battich et al., 2020), in which a variable initial kinetics experiment and later accompanying degradation experiment are conducted but kept in a fixed time period.


Estimating the parameters and RNA velocities with labeling data involves some technical subtleties, which we took into account when developing the corresponding algorithms tailored for each labeling strategy. Overall, we estimate absolute splicing and degradation constants ($\beta, \gamma$) by first estimating the degradation rates from labeling data and then the scaled degradation rate constant ($\hat{\gamma} = \gamma / \beta$) from splicing data, followed by obtaining a confident splicing rate constant (See MATERIALS AND METHODS and SUPPLEMENTARY METHODS from [Qiu, et. al](https://www.biorxiv.org/content/10.1101/696724v2) for details). For kinetics/chase experiments, we designed a two-step method (Fig. 2D-I from [Qiu, et. al](https://www.biorxiv.org/content/10.1101/696724v2)), in which we first compute the slope, $k$, for the labeled ($l$) and total ($r$) RNA for different labeling time period $t$, based on the linear regression  $l = k\gamma$ where the $k$  at a particular time follows $k(t) = (1-e^{-\gamma t})$ (see MATERIALS AND METHODS, Fig. 2E left, 2H from [Qiu, et. al](https://www.biorxiv.org/content/10.1101/696724v2). This leads to a second linear relationship between the labeling time $t$ and $-ln(1-k)$, whose slope is $\gamma$ (Fig. 2E right, 2I from [Qiu, et. al](https://www.biorxiv.org/content/10.1101/696724v2)).

<img src="https://user-images.githubusercontent.com/7456281/93839275-747fa500-fc5a-11ea-9605-24947534eacb.png">

The estimation framework described above is generally applicable to various data types, labeling strategies and biological systems, which also reconciles the programmable labeling kinetics and intrinsic splicing kinetics to absolute achieve accurate quantification of RNA synthesis and turnover rates, enabling functional biological discoveries.